In [1]:
import os
import pandas as pd
import numpy as np
import json
import folium
from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import branca.colormap as cm
from matplotlib import colors

In [2]:
df_refugee = pd.read_csv("time_series.csv",low_memory=False)
display(df_refugee.size)
df_refugee['Value'] = pd.to_numeric(df_refugee['Value'], errors='coerce')
df_refugee= df_refugee[df_refugee['Value'] > 0]
df_refugee = df_refugee[df_refugee['Year']>1984]
display(df_refugee.shape)
df_refugee = df_refugee.groupby(['Year','Country / territory of asylum/residence','Origin']).sum().reset_index()
df_refugee = df_refugee.rename(columns={"Year":"year","Value":"value"})
display(df_refugee.shape)

1492205

(161831, 5)

(111300, 4)

In [3]:
df_geo  = pd.read_csv("geo.csv",na_values ='NaN',keep_default_na =False)
display(df_geo.head())

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.93911,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [4]:
pd.DataFrame.to_csv(df_geo[['country', 'latitude', 'longitude']], "countries_latlon.csv", index=False)

In [5]:
df_merge = df_refugee.merge(df_geo, left_on='Country / territory of asylum/residence', right_on='name', how='inner')
df_merge = df_merge[['year','latitude','longitude','country','value','Origin']]
df_merge = df_merge.rename(columns={'latitude': 'latitude_asylum','longitude': 'longitude_asylum','country':'country_asylum'})
display(df_merge.head(10))

,year,latitude_asylum,longitude_asylum,country_asylum,value,Origin
0,1985,28.033886,1.659626,DZ,100.0,Chile
1,1985,28.033886,1.659626,DZ,2000.0,Palestinian
2,1985,28.033886,1.659626,DZ,7060.0,Various/Unknown
3,1985,28.033886,1.659626,DZ,165000.0,Western Sahara
4,1986,28.033886,1.659626,DZ,80.0,Chile
5,1986,28.033886,1.659626,DZ,2000.0,Palestinian
6,1986,28.033886,1.659626,DZ,140.0,Various/Unknown
7,1986,28.033886,1.659626,DZ,165000.0,Western Sahara
8,1987,28.033886,1.659626,DZ,2000.0,Palestinian
9,1987,28.033886,1.659626,DZ,165000.0,Western Sahara


In [6]:
df_merge = df_merge.merge(df_geo, left_on='Origin', right_on='name', how='inner')
display(df_merge.head())
df_merge = df_merge[['year','country','latitude','longitude','country_asylum','latitude_asylum','longitude_asylum','value']]
df_merge = df_merge.rename(columns={'latitude': 'latitude_origin','longitude': 'longitude_origin','country':'country_origin'})
df_merge = df_merge.sort_values("year")

df_merge = df_merge[df_merge['country_asylum']!=df_merge['country_origin']]

,year,latitude_asylum,longitude_asylum,country_asylum,value,Origin,country,latitude,longitude,name
0,1985,28.033886,1.659626,DZ,100.0,Chile,CL,-35.675147,-71.542969,Chile
1,1986,28.033886,1.659626,DZ,80.0,Chile,CL,-35.675147,-71.542969,Chile
2,1985,-38.416097,-63.616672,AR,80.0,Chile,CL,-35.675147,-71.542969,Chile
3,1986,-38.416097,-63.616672,AR,1690.0,Chile,CL,-35.675147,-71.542969,Chile
4,1993,-38.416097,-63.616672,AR,412.0,Chile,CL,-35.675147,-71.542969,Chile


In [20]:
display(df_merge.shape)

(106549, 8)

In [21]:
df_merge.to_csv("final_data.csv", sep=',', encoding='utf-8',index = False)